In [1]:
import os
import time
import logging
import subprocess
import unicodedata
import requests
from docx import Document
import chromadb
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# STEP 1: Check Ollama server
try:
    r = requests.get("http://localhost:11434")
    print("Ollama server is running! Status code:", r.status_code)
except Exception as e:
    print("Cannot reach Ollama server:", e)
    exit(1)

# STEP 2: Model config
config_list = [
    {
        "model": "gpt-oss:latest",
        "api_type": "ollama",
        "base_url": "http://localhost:11434",
        "stream": False,
    }
]

# STEP 3: Check model
def is_model_pulled(model_name="gpt-oss:latest"):
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        return model_name in result.stdout
    except Exception as e:
        print(f"Error checking model list: {e}")
        return False

if not is_model_pulled("gpt-oss:latest"):
    print("Model is NOT pulled.")
    exit(1)
print("Model is pulled and available.")

# STEP 4: Improved document loading
def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join(para.text.strip() for para in doc.paragraphs if para.text.strip())

docx_file = "Resume.docx"
if not os.path.exists(docx_file):
    print(f"Error: File {docx_file} not found")
    exit(1)

try:
    docs_content = extract_text_from_docx(docx_file)
    print("Preview:", docs_content[:300])
except Exception as e:
    print(f"Error loading document: {e}")
    exit(1)

# STEP 5: Setup logging
logging.basicConfig(
    filename='app.log',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# STEP 6: ChromaDB setup with retry logic
CHROMA_DIR = "./chroma_data"
max_retries = 3
retry_delay = 1

for attempt in range(max_retries):
    try:
        if os.path.exists(CHROMA_DIR):
            import shutil
            shutil.rmtree(CHROMA_DIR, ignore_errors=True)
            time.sleep(retry_delay)  # Give time for resources to be released
        
        chroma_client = chromadb.PersistentClient(path=CHROMA_DIR)
        break
    except Exception as e:
        if attempt == max_retries - 1:
            print(f"Failed to initialize ChromaDB after {max_retries} attempts: {e}")
            exit(1)
        time.sleep(retry_delay)

# STEP 7: Agent setup
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant that analyzes resumes and extracts skills.",
    llm_config={
        "config_list": config_list,
        "timeout": 600,
        "cache_seed": 42
    },
    code_execution_config=False  # Disable all code execution
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs": [docs_content],
        "chunk_token_size": 500,
        "model": "gpt-oss:latest",
        "collection_name": "resumes",
        "chroma_client": chroma_client,
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": True,
    },
    code_execution_config=False
)

# STEP 8: Improved chat initiation
qa_problem = """Analyze this resume and list all the technical skills mentioned. 
Format the response as a bullet point list categorizing skills as:
- Programming Languages
- Frameworks
- Tools/Platforms
- Other Technical Skills"""

try:
    print("\nStarting chat...")
    ragproxyagent.initiate_chat(
        assistant,
        message=qa_problem,
        silent=False
    )
except Exception as e:
    print(f"Chat failed: {e}")
    logging.exception("Chat failed")
    exit(1)

Ollama server is running! Status code: 200
Model is pulled and available.
Preview: Rathiga Ramesh
Full Stack Developer
County: Singapore 760116   Contact:88395846   email: rathigabecse@gmail.com
Professional Summary
Well-qualified Full Stack Developer familiar with a wide range of programming utilities and languages. Knowledgeable in backend and frontend development requirements a


2025-08-08 10:51:11,211 - autogen.agentchat.contrib.retrieve_user_proxy_agent - WARNING - docs_path is not provided in retrieve_config. Will raise ValueError if the collection `resumes` doesn't exist. Set docs_path to None to suppress this warning.
C:\Users\User\anaconda3\envs\ckd_env_clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Starting chat...
ragproxyagent (to assistant):

Analyze this resume and list all the technical skills mentioned. 
Format the response as a bullet point list categorizing skills as:
- Programming Languages
- Frameworks
- Tools/Platforms
- Other Technical Skills

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

I’m ready to help! Could you please paste the resume text or provide the details you’d like me to analyze? Once I have the information, I’ll extract and categorize all the technical skills for you.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (f69456d4-1f6e-4499-8727-2936a0a2ef12): Termination message condition on agent 'ragproxyagent' met


In [3]:
import os
import time
import logging
import requests
from docx import Document
import chromadb
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# 1. Configuration Setup
OLLAMA_URL = "http://localhost:11434"
CHROMA_DIR = "./chroma_data"
DOCX_FILE = "Resume.docx"

# 2. Verify Ollama
try:
    response = requests.get(OLLAMA_URL)
    print(f"Ollama server running (Status: {response.status_code})")
except Exception as e:
    print(f"Ollama connection failed: {e}")
    exit(1)

# 3. Load Resume with Error Handling
def load_resume(filepath):
    try:
        doc = Document(filepath)
        return "\n".join(para.text.strip() for para in doc.paragraphs if para.text.strip())
    except Exception as e:
        print(f"Error loading resume: {e}")
        exit(1)

if not os.path.exists(DOCX_FILE):
    print(f"Resume file not found: {DOCX_FILE}")
    exit(1)

resume_content = load_resume(DOCX_FILE)
print("Resume loaded successfully")

# 4. Initialize ChromaDB with Cleanup
def init_chroma():
    for _ in range(3):  # Retry logic
        try:
            if os.path.exists(CHROMA_DIR):
                import shutil
                shutil.rmtree(CHROMA_DIR, ignore_errors=True)
                time.sleep(1)  # Wait for lock release
            
            client = chromadb.PersistentClient(path=CHROMA_DIR)
            collection = client.get_or_create_collection("resumes")
            collection.add(
                documents=[resume_content],
                ids=["resume_1"]
            )
            return client
        except Exception as e:
            print(f"ChromaDB initialization attempt failed: {e}")
            time.sleep(2)
    print("Failed to initialize ChromaDB after multiple attempts")
    exit(1)

chroma_client = init_chroma()

# 5. Configure Agents
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a resume analysis expert. Extract and categorize all technical skills from resumes.",
    llm_config={
        "config_list": [{
            "model": "llama3.1:latest",
            "base_url": OLLAMA_URL,
            "api_type": "ollama"
        }],
        "timeout": 600
    },
    code_execution_config=False
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": None,  # Explicitly set to None
        "docs": [resume_content],
        "collection_name": "resumes",
        "chroma_client": chroma_client,
        "embedding_model": "all-mpnet-base-v2",
        "chunk_token_size": 500,
        "get_or_create": True
    },
    code_execution_config=False
)

# 6. Execute Analysis
try:
    print("\nAnalyzing resume...")
    ragproxyagent.initiate_chat(
        assistant,
        message="""Extract all technical skills from this resume and categorize them as:
        - Programming Languages
        - Frameworks/Libraries  
        - Tools/Platforms
        - Other Technical Skills
        Provide only the categorized list with no additional commentary.""",
        silent=False
    )
except Exception as e:
    print(f"Analysis failed: {e}")
    exit(1)

Ollama server running (Status: 200)
Resume loaded successfully

Analyzing resume...
ragproxyagent (to assistant):

Extract all technical skills from this resume and categorize them as:
        - Programming Languages
        - Frameworks/Libraries  
        - Tools/Platforms
        - Other Technical Skills
        Provide only the categorized list with no additional commentary.

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

Here is the extracted and categorized list of technical skills:

**Programming Languages**
- Java
- Python
- C++
- JavaScript
- SQL

**Frameworks/Libraries**
- Spring Boot
- Hibernate
- Django
- React
- Angular
- TensorFlow
- OpenCV

**Tools/Platforms**
- Jupyter Notebook
- IntelliJ IDEA
- Eclipse
- Android Studio
- AWS (Amazon Web Services)
- Azure
- Docker
- Kubernetes
- Git
- SVN

**Other Technical Skills**
- Data Analysis and Visualization (Tableau, Power BI)
- Machine Learning (Scikit-learn, PyT

ChatBot

In [ ]:
import os
import time
from docx import Document
import chromadb
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Configuration - Update these values
CONFIG = {
    "resume_file": "Resume.docx",
    "chroma_dir": "./chroma_data",
    "user_data": {
        "name": "Rathiga",
        "target_role": "AI Generalist",
        "experience": "8+ in IT",
        "location": "Singapore"
    },
    "ollama": {
        "base_url": "http://localhost:11434",
        "model": "llama3.1:latest"
        #"model": "gpt-oss:latest"  # Make sure this model is pulled in Ollama
    }
}

class LocalJobAssistant:
    def __init__(self):
        self.resume_content = self.load_resume()
        self.chroma_client = self.init_chroma()
        self.agents = self.create_agents()
        
    def load_resume(self):
        """Load and return resume text from DOCX file"""
        try:
            doc = Document(CONFIG['resume_file'])
            return "\n".join(para.text.strip() for para in doc.paragraphs if para.text.strip())
        except Exception as e:
            print(f"Error loading resume: {e}")
            exit(1)
    
    def init_chroma(self):
        """Initialize ChromaDB with retry logic"""
        for _ in range(3):  # Retry up to 3 times
            try:
                if os.path.exists(CONFIG['chroma_dir']):
                    import shutil
                    shutil.rmtree(CONFIG['chroma_dir'], ignore_errors=True)
                    time.sleep(1)
                
                client = chromadb.PersistentClient(path=CONFIG['chroma_dir'])
                collection = client.get_or_create_collection("resumes")
                collection.add(documents=[self.resume_content], ids=["resume_1"])
                return client
            except Exception as e:
                print(f"ChromaDB init attempt failed: {e}")
                time.sleep(2)
        print("Failed to initialize ChromaDB after multiple attempts")
        exit(1)
    
    def create_agents(self):
        """Create local LLM agents using Ollama"""
        # Disable all code execution capabilities
        no_exec_config = {
            "use_docker": False,
            "work_dir": None
        }
        
        llm_config = {
            "config_list": [{
                "model": CONFIG['ollama']['model'],
                "base_url": CONFIG['ollama']['base_url'],
                "api_type": "ollama"
            }],
            "timeout": 600
        }
        
        resume_agent = AssistantAgent(
            name="Resume_Expert",
            system_message=f"""You are a career advisor analyzing this resume:
            {self.resume_content[:1000]}...
            User profile: {CONFIG['user_data']}
            Provide specific, actionable advice.""",
            llm_config=llm_config,
            code_execution_config=False  # Disable code execution
        )
        
        chat_agent = AssistantAgent(
            name="Career_Chatbot",
            system_message=f"""You are a friendly job search assistant. Help with:
            - Job applications
            - Interview prep
            - Career growth
            User details: {CONFIG['user_data']}""",
            llm_config=llm_config,
            code_execution_config=False  # Disable code execution
        )
        
        proxy = RetrieveUserProxyAgent(
            name="Resume_Proxy",
            retrieve_config={
                "docs": [self.resume_content],
                "collection_name": "resumes",
                "chroma_client": self.chroma_client,
                "embedding_model": "local",  # Use local embeddings
                "get_or_create": True
            },
            human_input_mode="TERMINATE",
            llm_config=llm_config,
            code_execution_config=False  # Disable code execution
        )
        
        return {
            "resume": resume_agent,
            "chat": chat_agent,
            "proxy": proxy
        }
    
    def analyze_resume(self):
        """Analyze resume and return suggestions"""
        result = self.agents['proxy'].initiate_chat(
            self.agents['resume'],
            message="Analyze my resume and suggest improvements for my target role",
            silent=True
        )
        return result.summary
    
    def generate_cover_letter(self, job_description):
        """Generate personalized cover letter"""
        result = self.agents['proxy'].initiate_chat(
            self.agents['resume'],
            message=f"Generate a cover letter for this job:\n{job_description}",
            silent=True
        )
        return result.summary
    
    def chat(self, message):
        """Interactive career chat"""
        user_proxy = UserProxyAgent(
            "user_proxy", 
            code_execution_config=False  # Disable code execution
        )
        user_proxy.initiate_chat(
            self.agents['chat'],
            message=message,
            silent=True
        )
        return user_proxy.last_message()

def main_menu():
    """Command line interface"""
    try:
        assistant = LocalJobAssistant()
    except Exception as e:
        print(f"Failed to initialize assistant: {e}")
        return
    
    while True:
        print("\n===== Job Search Assistant =====")
        print("1. Analyze my resume")
        print("2. Generate cover letter")
        print("3. Chat about career advice")
        print("4. Exit")
        
        choice = input("\nSelect option (1-4): ").strip()
        
        if choice == '1':
            print("\nAnalyzing your resume...")
            analysis = assistant.analyze_resume()
            print("\n=== Resume Analysis ===")
            print(analysis)
            
        elif choice == '2':
            print("\nPaste the job description (press Enter then Ctrl+D to finish):")
            job_desc = []
            while True:
                try:
                    line = input()
                except EOFError:
                    break
                job_desc.append(line)
            job_desc = "\n".join(job_desc)
            
            if not job_desc.strip():
                print("No job description provided")
                continue
                
            print("\nGenerating cover letter...")
            cover = assistant.generate_cover_letter(job_desc)
            print("\n=== Cover Letter ===")
            print(cover)
            
        elif choice == '3':
            print("\nCareer Chatbot (type 'quit' to exit)")
            while True:
                user_input = input("\nYou: ").strip()
                if user_input.lower() == 'quit':
                    break
                response = assistant.chat(user_input)
                print(f"\nAssistant: {response}")
                
        elif choice == '4':
            print("Good luck with your job search!")
            break
            
        else:
            print("Invalid choice, please try again")

if __name__ == "__main__":
    # Verify Ollama connection first
    try:
        import requests
        response = requests.get(CONFIG['ollama']['base_url'], timeout=5)
        if response.status_code != 200:
            print("Ollama server not responding properly")
            exit(1)
    except Exception as e:
        print(f"Cannot connect to Ollama: {e}")
        exit(1)
    
    main_menu()

2025-08-08 11:31:24,941 - autogen.agentchat.contrib.retrieve_user_proxy_agent - WARNING - docs_path is not provided in retrieve_config. Will raise ValueError if the collection `resumes` doesn't exist. Set docs_path to None to suppress this warning.
C:\Users\User\anaconda3\envs\ckd_env_clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===== Job Search Assistant =====
1. Analyze my resume
2. Generate cover letter
3. Chat about career advice
4. Exit



Select option (1-4):  1



Analyzing your resume...


Please give feedback to Resume_Expert. Press enter or type 'exit' to stop the conversation:  Full stack GEN AI Developer
Please give feedback to Resume_Expert. Press enter or type 'exit' to stop the conversation:  2
Please give feedback to Resume_Expert. Press enter or type 'exit' to stop the conversation:  exit


...


Select option (1-4):  3



Career Chatbot (type 'quit' to exit)



You:  improve in AI ??
Replying as user_proxy. Provide feedback to Career_Chatbot. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (31b7f0e5-be16-4604-817f-a7cee43b1039): User requested to end the conversation

Assistant: {'content': 'As an AI Generalist, you\'ll be working on a wide range of projects that involve developing and implementing AI solutions. Here are some suggestions to improve your skills:\n\n**Online Courses:**\n\n1. **Coursera - Machine Learning Specialization**: A popular specialization by Andrew Ng that covers the basics of machine learning.\n2. **edX - AI for Everyone**: A course designed for non-technical professionals to understand AI concepts.\n3. **Stanford University on Stanford Online - Natural Language Processing with Deep Learning**: A course on NLP using deep learning techniques.\n\n**Certifications:**\n\n1. **Google Cloud Certified - Professional Machine Learning Engineer**: Demonstrate your expertise in machine learning on Google Cloud Platform.\n2. **Microsoft Certified: AI-900 Associate AI Analyst**: Showcase your skills in AI and machine learning on Micro


You:  exit
